<a href="https://colab.research.google.com/github/Churchiill/MilkywayFormation/blob/main/MilkyWay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pre

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io, requests, zipfile


# plt.rcParams.update({'font.size': 20, 'figsize':(8,6)})

SMALL_SIZE = 12
MEDIUM_SIZE = 15
BIGGER_SIZE = 18

plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

plt.rc('axes', linewidth = 5 )
plt.rc('lines', linewidth = 5 )

plt.rc('lines', markersize = 5 )

plt.rc('figure', figsize=(8,4) )         # Image size

# Import data

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


# **Convert Dat. files to a CSV Function**


In [25]:
import pandas as pd
from pathlib import Path
import csv

f = open("/content/draft.csv", "x")

def data2csv(path):
  files = Path(path).rglob("*.dat")

  i = 0

  for file in files:
    datContent = [i.strip().split() for i in open(file).readlines()]

    # write it as a new CSV file
    with open("/content/draft.csv", "w") as f:
      writer = csv.writer(f)
      writer.writerows(datContent)

    df = pd.read_csv("/content/draft.csv", names=['a','b','c']) 
    if i==1:
      data = pd.concat([data,df], axis=1)
    else:
      data = df 
    i = 1

  return data

# Converting data

In [ ]:
!pip install pyunpack
!pip install patool
from pyunpack import Archive
Archive('/content/drive/MyDrive/phot.tar.gz').extractall('/content/drive/MyDrive')

In [29]:
data = data2csv('/content/drive/MyDrive/phot/I')
data

,2171.88226,18.316,0.035,2165.84172,17.760,0.024,2165.85632,18.078,0.028,2183.87105,16.734,0.013,2166.83000,17.875,0.021,2166.83369,17.128,0.013,2166.86329,17.923,0.023,2166.86702,17.125,0.013,2166.84527,17.519,0.018,2166.84886,18.235,0.033,2166.86329,17.763,0.023,2166.85961,17.869,0.026,2166.86329,17.978,0.025,2166.84886,...,0.023,2187.83173,17.473,0.020,2177.79396,17.331,0.015,2187.85691,17.128,0.015,2187.85691,17.026,0.016,2177.76434,17.921,0.028,2187.85691,16.750,0.012,2187.86044,17.377,0.018,2187.86765,16.888,0.015,2177.76803,17.440,0.018,2177.77538,17.354,0.018,2194.87789,18.120,0.023,2188.81152,17.427,0.027,2188.81152,17.774,0.033
0,2183.84165,18.256,0.045,2171.86412,17.346,0.016,2165.88307,18.045,0.027,2189.81734,16.897,0.014,2172.87877,17.918,0.020,2172.88246,17.196,0.014,2189.86887,17.730,0.025,2189.87256,17.389,0.018,2172.89528,17.226,0.014,2172.89886,18.362,0.033,2189.86887,17.817,0.027,2185.82550,17.851,0.042,2189.86887,17.942,0.028,2172.89886,...,0.018,2194.79678,17.111,0.014,2187.84239,17.086,0.014,2194.82402,16.978,0.013,2194.82402,16.824,0.012,2187.87156,17.677,0.028,2194.82402,17.049,0.013,2194.82734,17.473,0.017,2194.83402,16.797,0.011,2187.87529,17.276,0.019,2187.88257,17.414,0.024,2211.79383,18.036,0.030,2211.80045,17.219,0.018,2211.80045,17.551,0.021
1,2184.84118,17.774,0.026,2183.82360,17.499,0.022,2171.87872,18.453,0.038,2191.84658,16.949,0.010,2189.83600,17.723,0.025,2189.83966,17.174,0.015,2192.77176,17.926,0.021,2192.77508,17.425,0.014,2183.88857,17.208,0.020,2183.89214,17.772,0.048,2192.77176,17.758,0.019,2189.86524,17.732,0.028,2192.77176,17.699,0.019,2183.89214,...,0.023,2206.80518,17.401,0.020,2194.81077,16.700,0.011,2206.83315,17.113,0.016,2206.83315,16.770,0.014,2188.81571,17.902,0.036,2206.83315,16.755,0.013,2206.83647,17.340,0.021,2206.84311,16.801,0.013,2188.81942,17.060,0.019,2188.82673,17.337,0.023,2221.82595,17.894,0.030,2242.72773,17.082,0.016,2221.83254,17.533,0.022
2,2191.81204,17.933,0.019,2184.82308,17.548,0.020,2171.90436,18.330,0.028,2200.85280,16.837,0.010,2191.87033,17.788,0.018,2200.87438,17.162,0.012,2201.78397,17.918,0.020,2201.78729,17.404,0.013,2189.85098,17.392,0.020,2189.85456,17.907,0.035,2201.78397,17.780,0.019,2192.76843,18.038,0.025,2201.78397,17.726,0.018,2189.85456,...,0.021,2215.85040,17.208,0.015,2206.81992,17.227,0.018,2216.82421,16.959,0.013,2216.82421,16.977,0.013,2194.84643,17.732,0.022,2216.82421,16.979,0.012,2216.82752,17.292,0.016,2216.83413,17.133,0.013,2194.84976,17.290,0.017,2194.85640,17.170,0.013,2225.83184,17.831,0.019,2250.71326,17.352,0.017,2225.83845,17.681,0.018
3,2200.82188,18.295,0.027,2191.78470,17.603,0.016,2183.83809,18.172,0.035,2211.82584,16.740,0.010,2200.87108,17.857,0.020,2212.78834,17.180,0.013,2212.81622,17.293,0.016,2212.81953,17.319,0.015,2192.75518,17.191,0.012,2192.75850,18.233,0.027,2212.81622,17.820,0.024,2201.77885,17.964,0.022,2212.81622,17.722,0.021,2192.75850,...,0.023,2225.73139,17.441,0.017,2215.86452,16.824,0.012,2225.75888,17.034,0.013,2225.75888,16.914,0.012,2206.84876,17.917,0.036,2225.75888,16.864,0.011,2225.76218,17.334,0.014,2225.76878,16.821,0.011,2211.76306,16.950,0.013,2211.76968,17.157,0.015,2227.85522,18.035,0.024,2275.85189,17.017,0.015,2227.86640,17.559,0.022
4,2206.75803,18.264,0.040,2200.80531,17.742,0.018,2183.86379,18.180,0.031,2217.75270,16.896,0.013,2212.78503,17.925,0.022,2217.77326,17.061,0.014,2217.80061,17.881,0.028,2217.80392,17.153,0.015,2201.76460,17.289,0.013,2201.76819,18.180,0.027,2217.80061,17.788,0.026,2212.81291,18.145,0.035,2217.80061,17.750,0.025,2201.76819,...,0.018,2227.80590,17.280,0.015,2216.81097,16.680,0.011,2225.78664,17.010,0.013,2225.78664,16.935,0.012,2211.75975,17.962,0.030,2225.78664,16.801,0.011,2225.78995,17.309,0.013,2225.79656,16.871,0.011,2221.79848,17.573,0.024,2221.80509,17.450,0.023,2229.86085,17.896,0.027,2282.85111,16.960,0.014,2229.86743,17.619,0.032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...